In [ ]:
!pip install kaggle kagglehub
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


print("Bibliotecas importadas")

In [ ]:
# Carregando o dataset
df = pd.read_csv('dataset.csv')
print("Dataset 'dataset.csv' carregado com sucesso!")

print("\nVisualização:")
display(df.head())

In [ ]:
df_processed = df.copy()

# A coluna TotalCharges tem alguns espaços vazios que precisam ser tratados.
df_processed['TotalCharges'] = pd.to_numeric(df_processed['TotalCharges'], errors='coerce')
median_value = df_processed['TotalCharges'].median() # Calcula a mediana da coluna
df_processed['TotalCharges'].fillna(median_value, inplace=True)

# Removendo a coluna customerID, pois não é relevante
df_processed.drop('customerID', axis=1, inplace=True)

# Separando as features (X) e a variável alvo (y)
X = df_processed.drop('Churn', axis=1)
y = df_processed['Churn'].apply(lambda x: 1 if x == 'Yes' else 0) # Convertendo 'Yes'/'No' para 1/0

# Identificando colunas numéricas e categóricas para o pré-processamento
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# transformador para aplicar OneHotEncoding nas colunas categóricas e StandardScaler nas colunas numéricas.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

X_processed = preprocessor.fit_transform(X)

# Dividindo os dados em conjuntos de treino e teste 80% treino, 20% teste
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Construção do modelo sequencial
model = keras.Sequential([
    keras.layers.Dense(units=32, activation='relu', input_shape=[X_train.shape[1]]),
    keras.layers.Dropout(0.3),

    # Camada oculta
    keras.layers.Dense(units=16, activation='relu'),
    keras.layers.Dropout(0.2),

    # A saída será uma probabilidade entre 0 e 1
    keras.layers.Dense(units=1, activation='sigmoid')
])

# Compilando modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Treino modelo
smote = SMOTE(random_state=42)

# Aplicamos o SMOTE APENAS nos dados de treino, não pode ser nos de validação ou de teste
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

history = model.fit(
    X_train_resampled,
    y_train_resampled,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

In [ ]:
# Avaliando o modelo com o conjunto de teste
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nAcurácia no conjunto de teste: {accuracy:.4f}")

y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten() # Convertendo probabilidades para 0 ou 1

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

print("\nMatriz de Confusão:")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.xlabel('Previsão')
plt.ylabel('Verdadeiro')
plt.show()

In [ ]:
# Exemplo de dados de um novo cliente.
novo_cliente_data = {
    'gender': ['Female'],
    'SeniorCitizen': [0],
    'Partner': ['Yes'],
    'Dependents': ['No'],
    'tenure': [1],
    'PhoneService': ['No'],
    'MultipleLines': ['No phone service'],
    'InternetService': ['Yes'],
    'OnlineSecurity': ['No'],
    'OnlineBackup': ['Yes'],
    'DeviceProtection': ['No'],
    'TechSupport': ['Yes'],
    'StreamingTV': ['Yes'],
    'StreamingMovies': ['No'],
    'Contract': ['Month-to-month'],
    'PaperlessBilling': ['Yes'],
    'PaymentMethod': ['Electronic check'],
    'MonthlyCharges': [29.85],
    'TotalCharges': [29.85]
}

novo_cliente_df = pd.DataFrame(novo_cliente_data)

# Aplicando o mesmo pré-processamento usado nos dados de treino
novo_cliente_processed = preprocessor.transform(novo_cliente_df)

previsao_proba = model.predict(novo_cliente_processed)

# Convertendo a probabilidade para 0 ou 1
previsao_churn = (previsao_proba > 0.5).astype(int).flatten()

if previsao_churn[0] == 1:
    print(f"A previsão de churn para este cliente é: Sim (Probabilidade: {previsao_proba[0][0]:.4f})")
else:
    print(f"A previsão de churn para este cliente é: Não (Probabilidade: {previsao_proba[0][0]:.4f})")